# Admissions Classifier

Binary classifier: predict Accepted vs Rejected using GPA, SAT_Score, Extracurricular_Activities.

In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from scaler import SimpleScaler

print(f"TensorFlow: {keras.__version__}")

## Load and preprocess data

In [ ]:
df = pd.read_csv("student_admission_dataset.csv")
df.head(10)

In [ ]:
# Binarize: Accepted=1, Rejected/Waitlisted=0
df["Admit"] = (df["Admission_Status"] == "Accepted").astype(int)

feature_cols = ["GPA", "SAT_Score", "Extracurricular_Activities"]
X = df[feature_cols].values.astype(np.float32)
y = df["Admit"].values

print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Class balance: Admit=1 {y.sum()}, Admit=0 {len(y) - y.sum()}")

In [ ]:
# Train/test split (80/20)
np.random.seed(42)
idx = np.random.permutation(len(X))
split = int(0.8 * len(X))
train_idx, test_idx = idx[:split], idx[split:]

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

In [ ]:
# Scale features
scaler = SimpleScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Build and train model

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(3,)),
    layers.Dense(8, activation="relu"),
    layers.Dense(1, activation="sigmoid"),
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=1)

## Evaluate

In [ ]:
loss, acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test accuracy: {acc:.4f}")

preds = (model.predict(X_test_scaled, verbose=0) > 0.5).astype(int).flatten()
print(f"Predictions: {preds[:20]}...")

## Save model and scaler

In [ ]:
import pickle

model.save("model.keras")
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)
print("Saved model.keras and scaler.pkl")